In [1]:
using Pkg
Pkg.activate(@__DIR__)
push!(LOAD_PATH, joinpath(@__DIR__, "..", ".."))
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [3]:
using JuMP
using BARON
using LinearAlgebra
using MINLPTrajOpt
using Rotations
using RigidBodyDynamics

┌ Info: Recompiling stale cache file /home/twan/.julia/compiled/v1.0/RigidBodyDynamics/WeevQ.ji for RigidBodyDynamics [366cf18f-59d5-5db9-a4de-86a9f6786172]
└ @ Base loading.jl:1184


In [4]:
urdf = joinpath(dirname(pathof(RigidBodyDynamics)), "..", "test", "urdf", "Acrobot.urdf")
mechanism = parse_urdf(urdf)

Spanning tree:
Vertex: world (root)
  Vertex: upper_link, Edge: shoulder
    Vertex: lower_link, Edge: elbow
No non-tree joints.

Design decisions:

* `Problem` type:
  * stores a `JuMP.Model`
  * does all the setup in the constructor
  * stores variables/useful subexpressions
  * solve method
  * methods for accessing results
  * use `AxisArrays` to store variables
* No separate `Result` type
* Implement `TrigPoly` type, run through `mass_matrix` and `dynamics_bias`.

Is this really needed?